In [1]:
!pip install xlwt openpyxl


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Recuperar la página web
url = "https://www.bloghemia.com/2019/05/noam-chomsky-michel-foucault-debate.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extraer el encabezado y armar el dataframe
header = soup.head
header_tags = [tag.name for tag in header.find_all()]
header_contents = [tag.string for tag in header.find_all()]

df_header = pd.DataFrame({'Tag': header_tags, 'Content': header_contents})

# Agrupar por tipo de etiqueta
grouped_df = df_header.groupby('Tag').agg(lambda x: ', '.join(x.dropna())).reset_index()

# Escribir los estilos en un archivo CSS
inline_styles = soup.find_all('style')
with open('Wrangle/styles.css', 'w') as f:
    for style in inline_styles:
        f.write(style.string)

/Users/jorgemagdaleno/PycharmProjects/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Paso 2: Eliminar todos los inline scripts y anuncios
for script in soup.find_all('script'):
    script.decompose()

for ad in soup.find_all(class_=lambda x: x and 'ad' in x):
    ad.decompose()

In [4]:
# Paso 3: Obtener todos los metadatos y escribirlos a un archivo Excel
meta_tags = soup.find_all('meta')
meta_data = [{'name': tag.get('name'), 'content': tag.get('content')} for tag in meta_tags]
df_meta = pd.DataFrame(meta_data)
df_meta.to_excel('Wrangle/metadata.xlsx', index=False)

In [11]:
import re

# Definir el patrón: que el texto inicie con ELDERS, CHOMSKY o FOUCAULT seguido de dos puntos.
pattern = re.compile(r'^(ELDERS|CHOMSKY|FOUCAULT):', re.IGNORECASE)

# Buscar todos los spans y filtrar aquellos que coincidan con el patrón
intervenciones = [span for span in soup.find_all('span') 
                   if pattern.match(span.get_text(strip=True))]

# Verificar la cantidad de intervenciones encontradas
print("Número de intervenciones encontradas:", len(intervenciones))

# Generar el HTML mínimo con las intervenciones
html_minimo = "<html><head><meta charset='UTF-8'><title>{}</title></head><body>{}</body></html>".format(
    soup.title.string if soup.title else "Debate", 
    '</br>'.join(str(intervencion) for intervencion in intervenciones)
)

with open('Wrangle/debate_minimo.html', 'w', encoding='utf-8') as f:
    f.write(html_minimo)

Número de intervenciones encontradas: 115


In [8]:

# Definir el patrón: el texto debe iniciar con "ELDERS", "CHOMSKY" o "FOUCAULT" seguido de dos puntos
pattern = re.compile(r'^(ELDERS|CHOMSKY|FOUCAULT):')

# Buscar todos los spans y filtrar los que cumplan con el patrón
intervenciones_spans = [
    span for span in soup.find_all('span')
    if pattern.match(span.get_text(strip=True))
]

# Preparar una lista para almacenar la información: nombre del interlocutor y su intervención
data = []
for span in intervenciones_spans:
    # Obtener el texto limpio del span
    texto = span.get_text(strip=True)
    # Separar el nombre del interlocutor y el contenido usando partition (divide en 3 partes)
    speaker, sep, content = texto.partition(':')
    data.append({
        "speaker": speaker.strip(),
        "intervention": content.strip()
    })

# Crear un DataFrame con la información
df_interv = pd.DataFrame(data)

# Agrupar las intervenciones por interlocutor y convertirlas en una lista
df_grouped = df_interv.groupby("speaker", as_index=False)["intervention"].agg(list)

# Guardar el DataFrame agrupado en un archivo JSON
df_grouped.to_json("Wrangle/interventions.json", orient="records", force_ascii=False, indent=2)
